In [1]:
# utils_pcn.py
import numpy as np
from sklearn.decomposition import PCA
from scipy.signal import convolve2d
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from torchvision.datasets import FashionMNIST
from torchvision import transforms

In [2]:
def load_fashion_mnist(n_train=10000, n_test=2000):
    transform = transforms.ToTensor()
    trainset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
    testset = FashionMNIST(root='./data', train=False, download=True, transform=transform)
    X_train = trainset.data[:n_train].numpy().astype(np.float32)
    y_train = np.array(trainset.targets[:n_train])
    X_test = testset.data[:n_test].numpy().astype(np.float32)
    y_test = np.array(testset.targets[:n_test])
    return X_train, y_train, X_test, y_test

In [3]:
def extract_random_patches(images, patch_size=7, stride=1, max_patches=50000):
    n, h, w = images.shape
    k = patch_size
    patch_vectors = []
    for idx in range(n):
        for i in range(0, h - k + 1, stride):
            for j in range(0, w - k + 1, stride):
                patch = images[idx, i:i + k, j:j + k]
                patch = patch - np.mean(patch)
                patch_vectors.append(patch.flatten())
                if len(patch_vectors) >= max_patches:
                    return np.array(patch_vectors).T
    return np.array(patch_vectors).T

In [4]:
def learn_pca_filters(patches, num_filters, patch_size):
    pca = PCA(n_components=num_filters)
    pca.fit(patches.T)
    filters = pca.components_.reshape((num_filters, patch_size, patch_size))
    return filters

In [5]:
def convolve_images(images, filters):
    feature_maps = []
    for f in filters:
        fmaps = [convolve2d(img, f, mode='valid') for img in images]
        feature_maps.append(np.stack(fmaps))
    return feature_maps

In [6]:
def binary_hashing(feature_maps):
    bin_stack = np.stack([(fm > 0).astype(np.uint8) for fm in feature_maps], axis=0)
    powers = 2 ** np.arange(bin_stack.shape[0])[::-1].reshape((-1, 1, 1, 1))
    hashed = np.sum(bin_stack * powers, axis=0)
    return hashed

In [7]:
def block_histogram(images, block_size=(7, 7), num_bins=256, overlap=0.5):
    n, h, w = images.shape
    bh, bw = block_size
    step_h = int(bh * (1 - overlap))
    step_w = int(bw * (1 - overlap))
    features = []
    for img in images:
        blocks = []
        for i in range(0, h - bh + 1, step_h):
            for j in range(0, w - bw + 1, step_w):
                block = img[i:i + bh, j:j + bw]
                hist, _ = np.histogram(block, bins=num_bins, range=(0, num_bins))
                blocks.extend(hist)
        features.append(np.array(blocks))
    return np.stack(features)

In [8]:
def train_and_evaluate(features_train, y_train, features_test, y_test):
    clf = LinearSVC(max_iter=10000)
    clf.fit(features_train, y_train)
    y_pred = clf.predict(features_test)
    return accuracy_score(y_test, y_pred)

In [ ]:
# Jalankan pipeline PCN untuk Fashion-MNIST
import warnings
warnings.filterwarnings('ignore')

print("\n📦 Loading Fashion-MNIST...")
X_train, y_train, X_test, y_test = load_fashion_mnist()

print("\n🧩 Stage-1 Patch Extraction...")
patches1 = extract_random_patches(X_train, patch_size=7, stride=1, max_patches=50000)

print("\n🔧 Stage-1 PCA Filter Training...")
filters1 = learn_pca_filters(patches1, num_filters=6, patch_size=7)

print("\n🎛️ Stage-1 Convolution...")
fmap1 = convolve_images(X_train, filters1)

print("\n🧠 Stage-2 Patch Extraction...")
fmap1_concat = np.concatenate(fmap1, axis=0)[:3000]
patches2 = extract_random_patches(fmap1_concat, patch_size=7, stride=1, max_patches=50000)

print("\n🔧 Stage-2 PCA Filter Training...")
filters2 = learn_pca_filters(patches2, num_filters=11, patch_size=7)

print("\n🎛️ Stage-2 Convolution...")
fmap2 = []
for fmap in fmap1:
    filtered = convolve_images(fmap, filters2)
    fmap2.append(filtered)
fmap2 = [f for group in fmap2 for f in group]

print("\n🔐 Binary Hashing + Histogram (Train)...")
hashed_train = binary_hashing(fmap2)
features_train = block_histogram(hashed_train, block_size=(7, 7), overlap=0.5)

print("\n🧪 Convolution on Test Set...")
fmap1_test = convolve_images(X_test, filters1)
fmap2_test = []
for fmap in fmap1_test:
    filtered = convolve_images(fmap, filters2)
    fmap2_test.append(filtered)
fmap2_test = [f for group in fmap2_test for f in group]

hashed_test = binary_hashing(fmap2_test)
features_test = block_histogram(hashed_test, block_size=(7, 7), overlap=0.5)

print("\n🏷️ Training + Evaluating Linear SVM...")
acc = train_and_evaluate(features_train, y_train, features_test, y_test)
print(f"\n✅ Final Test Accuracy on Fashion-MNIST: {acc * 100:.2f}%")



📦 Loading Fashion-MNIST...


100%|██████████| 26.4M/26.4M [04:12<00:00, 105kB/s] 
100%|██████████| 29.5k/29.5k [00:02<00:00, 12.5kB/s]
100%|██████████| 4.42M/4.42M [00:38<00:00, 116kB/s] 
100%|██████████| 5.15k/5.15k [00:00<00:00, 5.25MB/s]



🧩 Stage-1 Patch Extraction...

🔧 Stage-1 PCA Filter Training...

🎛️ Stage-1 Convolution...

🧠 Stage-2 Patch Extraction...

🔧 Stage-2 PCA Filter Training...

🎛️ Stage-2 Convolution...

🔐 Binary Hashing + Histogram (Train)...

🧪 Convolution on Test Set...

🏷️ Training + Evaluating Linear SVM...

✅ Final Test Accuracy on Fashion-MNIST: 10.35%


: 